In [1]:
# Imports
import duckdb
from pyspark.sql import SparkSession
import warnings
warnings.filterwarnings("ignore")

import sys
import os
current_dir = os.path.dirname(os.path.abspath('__file__'))
parent_dir = os.path.abspath(os.path.join(current_dir, os.pardir))

# Agregate the dic to sys.path
if parent_dir not in sys.path:
    sys.path.append(parent_dir)
import utils

## Connection to formatted database
jdbc_url = 'jdbc:duckdb:./../data/explotation_zone/barcelona_processed.db'
driver = "org.duckdb.DuckDBDriver"

# SparkSession inicialitzation
spark = SparkSession.builder\
    .config("spark.jars", "./../lib/duckdb.jar") \
    .appName("DataExplotation") \
    .getOrCreate()

24/06/04 16:58:31 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
df_airbnb = spark.read \
  .format("jdbc") \
  .option("url", jdbc_url) \
  .option("driver", driver) \
  .option("query", "SELECT * FROM df_airbnb_listings") \
  .load()

In [3]:
from pyspark.ml.feature import Bucketizer

In [4]:
bucketizer = Bucketizer(splits=[0, 50, 150, 250, float('Inf')],
                        inputCol='price',
                        outputCol='price_discretized')

In [5]:
df_airbnb = bucketizer.setHandleInvalid('keep').transform(df_airbnb)

In [7]:
df_airbnb = df_airbnb.drop('price')

In [8]:
df_criminal = spark.read \
  .format("jdbc") \
  .option("url", jdbc_url) \
  .option("driver", driver) \
  .option("query", "SELECT * FROM df_criminal_dataset") \
  .load()

In [9]:
df_reviews = spark.read \
  .format("jdbc") \
  .option("url", jdbc_url) \
  .option("driver", driver) \
  .option("query", "SELECT * FROM df_tripadvisor_reviews") \
  .load()

In [10]:
df_locations = spark.read \
  .format("jdbc") \
  .option("url", jdbc_url) \
  .option("driver", driver) \
  .option("query", "SELECT * FROM df_tripadvisor_locations") \
  .load()


In [11]:
con = duckdb.connect(database='./../data/explotation_zone/barcelona_processed_emb.db', read_only=False)
con.close()

## Write on the tables 
print('Writting tables...')
jdbc_url = 'jdbc:duckdb:./../data/explotation_zone/barcelona_processed_emb.db'
driver = "org.duckdb.DuckDBDriver"

print('    - Criminal table')
df_criminal.write \
    .format("jdbc") \
    .option("url", jdbc_url) \
    .option("dbtable", "df_criminal_dataset") \
    .option("driver", driver) \
    .mode("overwrite") \
    .save()

print('    - Aribnb table')
df_airbnb.write \
    .format("jdbc") \
    .option("url", jdbc_url) \
    .option("dbtable", "df_airbnb_listings") \
    .option("driver", driver) \
    .mode("overwrite") \
    .save()

print('    - Tripadvisor Locations table')
df_locations.write \
    .format("jdbc") \
    .option("url", jdbc_url) \
    .option("dbtable", "df_tripadvisor_locations") \
    .option("driver", driver) \
    .mode("overwrite") \
    .save()

print('    - Tripadvisor Reviews table')
df_reviews.write \
    .format("jdbc") \
    .option("url", jdbc_url) \
    .option("dbtable", "df_tripadvisor_reviews") \
    .option("driver", driver) \
    .mode("overwrite") \
    .save()

spark.stop()
con.close()

Writting tables...
    - Criminal table


    - Aribnb table


24/06/04 16:59:26 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


    - Tripadvisor Locations table


    - Tripadvisor Reviews table
